# Movies Recommend System

### 構成

1. レビューの形態素解析
2. 特徴量行列の作成
3. 事前に作成したモデルで予測

### 準備

In [132]:
#パンダスの準備
import pandas as pd
from pandas import Series,DataFrame

In [133]:
#SQLの準備
import sqlite3
con = sqlite3.connect("movies.db")

In [134]:
# 関数を作って、SQL文の実行結果をDataFrameにして返します。
def sql_to_df(sql_query):

    # read_sqlの引数に、SQL文とデータベースへのConnectionを渡します。
    df = pd.read_sql(sql_query, con)

    # 結果のDataFrameを返します。
    return df

In [135]:
import MeCab
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

## 1. レビューの形態素解析

In [139]:
watch = '''
コメディ映画。めちゃくちゃ笑いたい。イギリスっぽいアイロニーを含んだ映画がみたい。
知的な笑いで、パティントンとかそういう映画。
テッドとか下品な映画ではなくて、上品な笑いがいい。
'''

In [140]:
text = watch
mecab.parse('')#文字列がGCされるのを防ぐ
node = mecab.parseToNode(text)
i_id=0
while node:
    #単語を取得
    word = node.surface
    #品詞を取得
    pos = node.feature.split(",")[1]
    #print('{}'.format(word))
    #print('{0} , {1}'.format(word, pos))
        
    #次の単語に進める
    node = node.next
    
    sql_query = '''
    INSERT INTO watch_cnt_table(id, word, num) values({i_id}, "{word}", 1)
    '''.format(i_id=i_id, word=word)
        
    con.execute(sql_query)
    
i_id += 1

##### 事前に作成しているテーブル

CREATE TABLE watch_cnt_table(
    id int,
    title char(50),
    word char(30),
    num int
);

In [105]:
sql_query = '''
SELECT * FROM watch_cnt_table
'''
words_df = sql_to_df(sql_query)

In [106]:
words_df.tail()

,id,word,num
40,0,笑い,1
41,0,が,1
42,0,いい,1
43,0,。,1
44,0,,1


In [14]:
#sql_query = '''
#DELETE  FROM watch_cnt_table
#'''
#con.execute(sql_query)

In [ ]:
#con.close()

## 2. 特徴量行列の作成

In [141]:
i = 1
words_cnt = []

In [136]:
#The first step —————————————————————————————————————————————————
#全映画で使われたwordをとりだす。
sql_query = '''
SELECT word FROM cnt_table WHERE word not like '%''%' GROUP BY word ORDER BY sum(num) desc
'''
words_used_df = sql_to_df(sql_query)

In [137]:
words_used_df.tail(1)

,word
31566,￣*


In [138]:
31566 / 1900

16.613684210526316

In [142]:
query_all = []

n = 0

for num in range(1,17):     
    query = '''SELECT '''
    for item in range(len(words_used_df)):
        if n <= num*1900:
            #レビューIDごとに出現しているwordでカウントする。
            query += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{n}, '''.format(word=words_used_df.iloc[item, 0], n=n)
        
            n += 1
    
            if n == num*1900:
                break
    
    query += '''id FROM watch_cnt_table GROUP BY id'''
    #print(query)
    query_all.append(query)

In [143]:
query = '''
SELECT distinct id FROM watch_cnt_table
'''
id_df = sql_to_df(query)

In [144]:
for num in range(1,17):
    print(num)
    res_df = sql_to_df(query_all[num-1])
    if num == 1:
        all_df = pd.merge(id_df, res_df, on='id')
    elif num > 1:
        all_df = pd.merge(all_df, res_df, on='id')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [145]:
all_df = all_df.drop('id', axis=1)

In [146]:
all_df

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a30390,a30391,a30392,a30393,a30394,a30395,a30396,a30397,a30398,a30399
0,0,5,2,2,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
con.close

<function Connection.close>

## 3. 事前に作成したモデルで予測

In [147]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [148]:
from sklearn import linear_model

参考 : http://qiita.com/yshi12/items/9f3f8ae69588da9f018f,  http://techblog.yahoo.co.jp/datascience/use_scikit-learn/

scikit-learnを使った多クラス分類


In [149]:
from sklearn.externals import joblib
#モデルの読み込み
logreg = joblib.load("model_dir/model")

In [150]:
all_numpyMatrix_data = all_df.as_matrix()
X_test = all_numpyMatrix_data
_test_data = DataFrame(X_test)

In [151]:
# テストデータを予測します。
Y_pred = logreg.predict(X_test)

In [152]:
Y_pred

array(['アダムス・ファミリー'], dtype=object)

In [127]:
watch[1:91]

'コメディ映画。めちゃくちゃ笑いたい。イギリスっぽいアイロニーを含んだ映画がみたい。\n知的な笑いで、パティントンとかそういう映画。\nテッドとか下品な映画ではなくて、上品な笑いがいい。'

In [128]:
pred = Y_pred[0]

In [122]:
pred

'アダムス・ファミリー'

In [129]:
query = '''
INSERT INTO want(watn, pred) values('{watch}', '{pred}')
'''.format(watch=watch[1:91], pred=pred)

In [ ]:
con.execute(query)

In [123]:
sql_query = '''
SELECT * FROM want
'''
want_df = sql_to_df(sql_query)

In [124]:
want_df

,id,watn,pred
0,None,爽やかな映画。若者が活躍する映画,グッド・ウィル・ハンティング／旅立ち
1,None,そこはかとなく暗くどん底の映画。,エド・ウッド


In [131]:
con.close()

In [130]:
query

"\nINSERT INTO want(watn, pred) values('コメディ映画。めちゃくちゃ笑いたい。イギリスっぽいアイロニーを含んだ映画がみたい。\n知的な笑いで、パティントンとかそういう映画。\nテッドとか下品な映画ではなくて、上品な笑いがいい。', 'アダムス・ファミリー')\n"